In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [28]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [29]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t) 
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)

        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)

        return t

In [30]:
#Turn off Computation Graph... turn off derivatives for now
torch.set_grad_enabled(False)

In [31]:
network = Network()

In [32]:
sample = next(iter(train_set))

In [33]:
image, label = sample
image.shape

torch.Size([1, 28, 28])

In [34]:
image.unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [35]:
pred = network(image.unsqueeze(0)) # image shape needs to be (batch_size x in_channels x H x W)

In [36]:
pred.shape

torch.Size([1, 10])

In [37]:
pred

tensor([[ 0.0897, -0.1394, -0.0378, -0.0537, -0.0450, -0.1002, -0.0221,  0.0876,  0.0328,  0.0414]])

In [38]:
label #This is what it is suppose to be

9

In [39]:
pred.argmax(dim=1)

tensor([0])

In [40]:
F.softmax(pred, dim=1)

tensor([[0.1107, 0.0880, 0.0975, 0.0959, 0.0968, 0.0916, 0.0990, 0.1105, 0.1046, 0.1055]])

In [41]:
F.softmax(pred, dim=1).sum()

tensor(1.)

In [42]:
#Predictions will be different everytime because the weights are always randomly generated
net1 = Network()

In [44]:
net1(image.unsqueeze(0))

tensor([[-0.0325, -0.1364,  0.1153, -0.0409, -0.0352,  0.1225,  0.0065, -0.0934,  0.0031, -0.1236]])

In [45]:
net2 = Network()

In [46]:
net2(image.unsqueeze(0))

tensor([[ 0.1253, -0.0765,  0.0936, -0.0356,  0.0584,  0.0271, -0.0837,  0.0815, -0.0300,  0.0051]])